Momentum investing is investing in stocks that have increased in price the most. 

Here, we will select the 50 stocks with highest price momentum. Then, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks. (From project 1)

In [3]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from datetime import datetime
import yfinance as yf

In [18]:
# pandas read html
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# clean data
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = datetime.today().strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*2)

# stacking the data to make it easier to work with
# use future_stack=True to avoid the future warning
df = yf.download(tickers=symbols_list, 
                 start=start_date, 
                 end=end_date).stack(future_stack=True)

df

[*********************100%%**********************]  503 of 503 completed


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2022-08-01 A       131.583359  133.429993  135.229996  133.259995  133.509995   
           AAL      14.280000   14.280000   14.320000   13.520000   13.670000   
           AAPL    159.703156  161.509995  163.589996  160.889999  161.009995   
           ABBV    129.740509  140.220001  142.839996  139.149994  141.509995   
           ABNB    111.199997  111.199997  113.959999  107.480003  110.000000   
...                       ...         ...         ...         ...         ...   
2024-07-26 XYL     140.839996  140.839996  142.130005  137.820007  138.479996   
           YUM     128.050003  128.050003  129.039993  127.410004  127.690002   
           ZBH     111.290001  111.290001  112.279999  110.230003  110.790001   
           ZBRA    325.980011  325.980011  330.970001  323.000000  326.500000   
           ZTS     179.839996  179.839996  183.850006  179.259995  182.259995   

Price                  Volume  
Date       Ticker              
2022-08-01 A        1081700.0  
           AAL     32730800.0  
           AAPL    67829400.0  
           ABBV     8523900.0  
           ABNB     6019500.0  
...                       ...  
2024-07-26 XYL      1074100.0  
           YUM      1874400.0  
           ZBH      1399400.0  
           ZBRA      458700.0  
           ZTS      2437300.0  

[251500 rows x 6 columns]

In [5]:
# we want to pull price in 1 yr stock return
tickers_list = df.index.get_level_values(1).unique()
AllYF = yf.Tickers(' '.join(tickers_list)).tickers
# this is a replacement for year1ChangePercent
# yeet = AllYF['AAPL'].info['52WeekChange']

In [14]:
df_columns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy']

data_list = [
    [
        ticker,
        values.info.get('previousClose', None), 
        values.info.get('52WeekChange', None),
        'N/A'
    ]
    for ticker, values in AllYF.items()
]

final_df = pd.DataFrame(data=data_list, columns=df_columns)
# 2:02

In [16]:
# inplace will directly modify final_df
final_df.sort_values(by='One-Year Price Return', ascending=False, inplace=True)
# get the top 50 results (50 highest returns)
final_df = final_df[:50]
# reset the index to start from 0
final_df.reset_index(drop=True, inplace=True)

Ngl, this is really lame. We will now create a more realistic momentum strategy with 1, 3, 6, and 12 month returns.

- High-quality m-stocks show "slow and steady" outperformance over long periods of time.
- Low-quality m-stocks might not show any momentum for a long time, then surge upwards.
